# CPU Virtualization

The job of the CPU is to run many more programs than we have processors in a computer.

N (# programs) >> M (# CPUS)

From each program's perspective, it looks like they each have their own individual CPU. 

Key Abstraction: **The Process** (a running program)

For now, we will assume they are single threaded.

A process has two major technical components:
1) A set of Registers
   - general purpose registers 
   - process specific registers
     - **instruction pointer** aka **program counter**
     - memory address registers
2) It's private address space

<img src="images/process_address_space.png" width="150">

How do we virtualize the CPU?

The simplest way, when a user wants to run a program, load it onto CPU and simply run it. This is **direct execution**.

<img src="images/lde.png" width="500">

Three major problems with simple direct execution:
1) What if the OS wants to swap from one process to another?
2) What if the program wants to perform some restricted operation?
3) What if the program performs some slow operation, like an IO request?

## Problem 2: Restricted Operations

Restricted OPs are any hardware access.

If we want to perform a restricted op, we must make a **system call**. A **system call** is an OS request.

The CPU has two modes: **Kernel (Priviledge) Mode** and **User (Unpriviledged) Mode**.

There is a literal dedicated piece of circuitry on the CPU which determines which mode the computer is currently in.

When a user process makes a system call, the OS must switch over to Kernel Mode and then execute the code for the system call.

When a system call is made, the computer **traps** down to the operating system. 

When a **trap** is executed:
1) the CPU is switched kernel mod
2) a **trap handler** is executed for the system call

A trap handler looks up in a **trap table** where the code for the system call actually is in memory and executes it.

The trap table is hidden from user processes. The trap table is initialized at system boot.

Once the system call completes, the OS will **return-from-trap** to give the CPU back to the user process.

<img src="images/lde_traps.png" width="500">

## Problem 1: Switching between processes

To switch between two processes, a **context switch** is performed.

In a context switch, the registers of the running process are stored in kernel memory, and the registers of the to-be-running process are loaded onto the CPU.

Processes can have different states:
1) RUNNING
2) READY
3) BLOCKED

<img src="images/context_switch.png" width="500">

## Problem 3: Slow Ops

Some operations, like file I/O are super slow. When you want to write to a file, the OS asks the hard disk to perform that write. The harddisk will evetually get back to OS saying that the write is complete.

If the current process stays on the CPU while this write is happening, millions of cpu cycles are wasted.

When an IO request is made, the OS will switch the current process (put into the BLOCKED state until the IO finishes) off of the CPU to allow some other process to run.

CPU Virtualization is accomplished through **Limited Direct Execution**. 